# **Day 4: Scratchcards**
This one seems pretty simple at first glance! Shouldn't be too tough. 

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

/Users/thubbard/Documents/Personal/Programming/advent-of-code-2023


Now, I'm going to import some libraries:

In [47]:
# Import statements
import pandas as pd
from copy import deepcopy

Finally, I'll load in the data for this puzzle. 

In [4]:
# Load in the data for the puzzle
with open("data/input-files/day-04-input.txt", "r") as txt_file:
    input_data = txt_file.readlines()

# Identifying Winning Numbers
Actually identifying the winning numbers in each of these games seems pretty trivial. I have a feeling the numbers themselves are going to come back in Part 2, so I'll keep track of which numbers are actually winning. 

In [29]:
def parse_input_into_dataframe(input_data):
    """
    This method will parse the puzzle's input_data into a DataFrame that contains various 
    pieces of info about the game. 
    """
    
    # Parse the ID, winning numbers, and ticket numbers from each of the lines into a DataFrame
    scratchcards_df = pd.DataFrame.from_records(
        [
            {
                "card_number": int(line.split(":")[0].split(" ")[-1]),
                "winning_numbers": [
                    int(x.strip())
                    for x in line.split(":")[-1].strip().split("|")[0].strip().split(" ")
                    if x != ""
                ],
                "ticket_numbers": [
                    int(x.strip())
                    for x in line.split(":")[-1].strip().split("|")[-1].strip().split(" ")
                    if x != ""
                ],
            }
            for line in input_data
        ]
    )

    # Determine which ticket_numbers are winning_numbers
    scratchcards_df["winning_ticket_numbers"] = scratchcards_df.apply(
        lambda row: [
            ticket_num
            for ticket_num in row.ticket_numbers
            if ticket_num in row.winning_numbers
        ],
        axis=1,
    )
    
    # Determine how many winning ticket numbers there are 
    scratchcards_df["n_winning_numbers"] = scratchcards_df["winning_ticket_numbers"].apply(
        lambda x: len(x)
    )

    # Determine the point value of each of the cards
    scratchcards_df["point_value"] = scratchcards_df["winning_ticket_numbers"].apply(
        lambda winning_numbers_list: (2 ** (len(winning_numbers_list) - 1))
        if len(winning_numbers_list) >= 1
        else 0
    )
    
    # Return the DataFrame
    return scratchcards_df

With this method in hand, we can calculate the sum of the values of the Elf's cards:

In [30]:
# Parse the input_data using the method we'd created
scratchcards_df = parse_input_into_dataframe(input_data)

# Calculate the sum of the point_values, and then print it
point_value_sum = scratchcards_df["point_value"].sum()
print(f"The sum of the values of the scratchcards is '{point_value_sum}'")

The sum of the values of the scratchcards is '25010'


# Part 2 - Duplicating Scratchcards
This one seems a little bit more challenging - I think that I need to basically iterate through the list of games (like the example shows), and build up a list of the different cards as I continue through it. 

In [49]:
# We'll declare some variables to keep track of some of the different game winnings as we iterate through
cur_card_number = 1
card_number_to_cards = {
    card_dict.get("card_number"): [card_dict]
    for card_dict in scratchcards_df.to_dict(orient="records")
}
original_cards_by_card_number = {
    card_dict.get("card_number"): card_dict
    for card_dict in scratchcards_df.to_dict(orient="records")
}

# Iterate through each of the cards
while cur_card_number <= scratchcards_df["card_number"].max():
    # Iterate through each of the cards associated with this current number
    for card_info in card_number_to_cards.get(cur_card_number, []):
        # Determine how many winning numbers there are in this ticket
        cur_card_n_winning_numbers = card_info.get("n_winning_numbers", 0)

        # Iterate through each of the following cards and add to the card_number_to_cards list
        for idx in range(cur_card_n_winning_numbers):
            next_card_num = (idx + 1) + cur_card_number
            card_number_to_cards[next_card_num].append(original_cards_by_card_number.get(next_card_num))

    # Once we're finished processing this card, we're going to iterate the cur_card_number
    cur_card_number += 1

Now that we've duplicated each of the cards, I'm going to make a quick DataFrame showing off how much of each card we have. 

In [52]:
# Create a DataFrame showing off the cards we have 
duplicated_card_ct_df = pd.DataFrame.from_records(
    [
        {"card_num": card_num, "n_cards": len(list_of_cards)}
        for card_num, list_of_cards in card_number_to_cards.items()
    ]
)

# Show off this DataFrame
duplicated_card_ct_df

,card_num,n_cards
0,1,1
1,2,2
2,3,4
3,4,8
4,5,16
...,...,...
207,208,29267
208,209,52938
209,210,52938
210,211,73745


Finally: how many cards did we end up with? 

In [53]:
# Print the total amount of cards we've ended up with
n_total_cards = duplicated_card_ct_df["n_cards"].sum()
print(f"After duplicating all of the cards, we end up with '{n_total_cards}' cards.")

After duplicating all of the cards, we end up with '9924412' cards.
